In [ ]:
!git clone https://github.com/yxccxxx/Thin-Plate-Spline-Motion-Model.git

Cloning into 'Thin-Plate-Spline-Motion-Model'...
remote: Enumerating objects: 1147, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 1147 (delta 58), reused 58 (delta 45), pack-reused 1056
Receiving objects: 100% (1147/1147), 233.97 MiB | 18.19 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Updating files: 100% (1035/1035), done.


In [ ]:
cd Thin-Plate-Spline-Motion-Model

/content/Thin-Plate-Spline-Motion-Model


In [ ]:
!mkdir checkpoints
!wget -c https://cloud.tsinghua.edu.cn/f/cd411b334a2e49cdb1e2/?dl=1 -O checkpoints/mgif.pth.tar

mkdir: cannot create directory ‘checkpoints’: File exists
--2023-05-12 01:49:58--  https://cloud.tsinghua.edu.cn/f/cd411b334a2e49cdb1e2/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/9c463bc4-a654-4a1f-8940-0c43add7af58/mgif.pth.tar [following]
--2023-05-12 01:50:00--  https://cloud.tsinghua.edu.cn/seafhttp/files/9c463bc4-a654-4a1f-8940-0c43add7af58/mgif.pth.tar
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 306156401 (292M) [application/octet-stream]
Saving to: ‘checkpoints/mgif.pth.tar’

checkpoints/mgif.pt 100%[===================>] 291.97M  6.72MB/s    in 34s     

2023-05-12 01:50:34 (8.68 MB/s) - ‘checkpoints/mgif.pth.tar’ saved [306156401/306156401]



In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 35.6 MB/s eta 0:00:00
   

# Train model on top of pre-trained model weights
--checkpoint: path to checkpoint to restore

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 python run.py --config config/mgif-256.yaml --checkpoint checkpoints/mgif.pth.tar --device_ids 0,1

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
None
Use predefined train-test split.
Training...
load success: -1
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a we

# Evaluation

In [ ]:
from frames_dataset import FramesDataset
from demo import load_checkpoints
import os
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import imageio
import yaml
from modules.inpainting_network import InpaintingNetwork
from modules.bg_motion_predictor import BGMotionPredictor
from modules.keypoint_detector import KPDetector
from modules.dense_motion import DenseMotionNetwork
from modules.avd_network import AVDNetwork

def load_model_from_checkpoint(config, checkpoint_path):
    inpainting_network = InpaintingNetwork(**config['model_params']['generator_params'],
                                           **config['model_params']['common_params'])

    if torch.cuda.is_available():
        inpainting_network.to(device)

    kp_detector = KPDetector(**config['model_params']['common_params'])
    dense_motion_network = DenseMotionNetwork(**config['model_params']['common_params'],
                                              **config['model_params']['dense_motion_params'])
                                                           
    if torch.cuda.is_available():
        kp_detector.to(device)
        dense_motion_network.to(device)

    bg_predictor = None
    if (config['model_params']['common_params']['bg']):
        bg_predictor = BGMotionPredictor()
        if torch.cuda.is_available():
            bg_predictor.to(device)
    
    checkpoint = torch.load(checkpoint_path)
    if inpainting_network is not None:
        inpainting_network.load_state_dict(checkpoint['inpainting_network'])
    if kp_detector is not None:
        kp_detector.load_state_dict(checkpoint['kp_detector'])
    if bg_predictor is not None and 'bg_predictor' in checkpoint:
        bg_predictor.load_state_dict(checkpoint['bg_predictor'])
    if dense_motion_network is not None:
        dense_motion_network.load_state_dict(checkpoint['dense_motion_network'])

    return inpainting_network, kp_detector, bg_predictor, dense_motion_network 

def load_dataset(mode, config_path):
    with open(config_path) as f:
        config = yaml.safe_load(f)
    dataset = FramesDataset(is_train=(mode == 'train'), **config['dataset_params'])
    return dataset

def reconstruction(config_path, inpainting_network, kp_detector, bg_predictor, dense_motion_network, dataset):
    with open(config_path) as f:
        config = yaml.safe_load(f)
    log_dir = os.path.join('checkpoints')
    png_dir = os.path.join(log_dir, 'reconstruction/png')
    log_dir = os.path.join(log_dir, 'reconstruction')

    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    if not os.path.exists(png_dir):
        os.makedirs(png_dir)
    
    loss_list = []
    test_loss_per_video = []

    inpainting_network.eval()
    kp_detector.eval()
    dense_motion_network.eval()
    if bg_predictor:
        bg_predictor.eval()

    for it, x in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            predictions = []
            visualizations = []
            if torch.cuda.is_available():
                x['video'] = x['video'].cuda()
            kp_source = kp_detector(x['video'][:, :, 0])
            test_loss = 0
            for frame_idx in range(x['video'].shape[2]):
                source = x['video'][:, :, 0]
                driving = x['video'][:, :, frame_idx]
                kp_driving = kp_detector(driving)
                bg_params = None
                if bg_predictor:
                    bg_params = bg_predictor(source, driving)
                
                dense_motion = dense_motion_network(source_image=source, kp_driving=kp_driving,
                                                    kp_source=kp_source, bg_param = bg_params, 
                                                    dropout_flag = False)
                out = inpainting_network(source, dense_motion)
                out['kp_source'] = kp_source
                out['kp_driving'] = kp_driving

                predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])
                loss = torch.abs(out['prediction'] - driving).mean().cpu().numpy()
                
                loss_list.append(loss)
                test_loss += loss
            # print(np.mean(loss_list))
            test_loss_per_video.append(test_loss/x['video'].shape[2])
            predictions = np.concatenate(predictions, axis=1)
            imageio.imsave(os.path.join(png_dir, x['name'][0] + '.png'), (255 * predictions).astype(np.uint8))
    print("\nReconstruction L1 loss: %s" % np.mean(loss_list))
    return test_loss_per_video

In [ ]:
device = torch.device('cuda:0')
config_path = 'config/mgif-256.yaml'
with open(config_path) as f:
    config = yaml.safe_load(f)

In [ ]:
# Evaluate model performance with original weights for 20 epoch
checkpoint_path = 'checkpoints/00000019-checkpoint.pth.tar'
inpainting_network, kp_detector, bg_predictor, dense_motion_network = load_model_from_checkpoint(config, checkpoint_path)
dataset = load_dataset('reconstruction', config_path)
# Reconstructed videos will be saved to ./checkpoints/reconstruction/png
test_loss = reconstruction(config_path, inpainting_network, kp_detector, bg_predictor, dense_motion_network, dataset)

None
Use predefined train-test split.


100it [01:41,  1.01s/it]


Reconstruction L1 loss: 0.021064786


# Tune hyperparameters

In [ ]:
 !CUDA_VISIBLE_DEVICES=0,1 python run.py --config config/mgif-256.yaml --checkpoint checkpoints/mgif.pth.tar --device_ids 0,1

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
None
Use predefined train-test split.
Training...
load success: -1
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a we

In [ ]:
checkpoint_path = 'checkpoints/00000019-checkpoint.pth.tar'
inpainting_network, kp_detector, bg_predictor, dense_motion_network = load_model_from_checkpoint(config, checkpoint_path)
dataset = load_dataset('reconstruction', config_path)
# Reconstructed videos will be saved to ./checkpoints/reconstruction/png
test_loss = reconstruction(config_path, inpainting_network, kp_detector, bg_predictor, dense_motion_network, dataset)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


None
Use predefined train-test split.


100it [01:40,  1.01s/it]


Reconstruction L1 loss: 0.020449812


In [ ]:
!nvidia-smi

Fri May 12 01:46:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
 !CUDA_VISIBLE_DEVICES=0,1 python run.py --config config/mgif-256.yaml --checkpoint checkpoints/mgif.pth.tar --device_ids 0,1

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
None
Use predefined train-test split.
Training...
load success: -1
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a we

In [ ]:
checkpoint_path = 'checkpoints/00000019-checkpoint.pth.tar'
inpainting_network, kp_detector, bg_predictor, dense_motion_network = load_model_from_checkpoint(config, checkpoint_path)
dataset = load_dataset('reconstruction', config_path)
# Reconstructed videos will be saved to ./checkpoints/reconstruction/png
vox_test_loss = reconstruction(config_path, inpainting_network, kp_detector, bg_predictor, dense_motion_network, dataset)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


None
Use predefined train-test split.


100it [01:55,  1.15s/it]


Reconstruction L1 loss: 0.02052473


# Git operations

In [ ]:
!git config --global user.email "yixincindy19@gmail.com"
!git config --global user.name "yxccxxx"

In [ ]:
!git remote -v

In [ ]:
# use personal access tokens from https://github.com/settings/tokens
!git remote set-url origin https://{access_token}@github.com/yxccxxx/Thin-Plate-Spline-Motion-Model.git

In [ ]:
!git pull

Already up to date.


In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    checkpoints/00000000-checkpoint.pth.tar

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add -A
!git commit -m "update from colab"

[main 36741a1] update from colab
 1 file changed, 0 insertions(+), 0 deletions(-)
 delete mode 100644 checkpoints/00000000-checkpoint.pth.tar


In [ ]:
!git push

Enumerating objects: 14, done.
Counting objects: 100% (14/14), done.
Delta compression using up to 12 threads
Compressing objects: 100% (10/10), done.
^C
